In [1]:
# Parameters
msgs = "Ran from Airflow at 2022-07-09T04:00:00+00:00!"


In [2]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver',chrome_options=chrome_options)

/data/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [3]:
import time
import os
import requests
import urllib.request
import urllib.parse
import json
import re
import collections
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import dateparser
from datetime import datetime
import numpy as np

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [4]:
ID = '002'
SITE ="leral.net"

In [5]:
categories = ["Politique_r2","Actualites_r1","Sport_r3","Television-Leral_r7","People_r41"]

In [6]:
def parseDate(date):
    try:
        return dateparser.parse(date, languages=['fr'])
    except:
        return dateparser.parse(date)

In [7]:
def getWebPage(URL):
    try:
        page  = requests.get(URL)
        return BeautifulSoup(page.text, "html.parser")
    except: print(URL)

In [8]:
def getArticleTitle(HTML):       
    h = HTML.find_all("div", {"class": "titre"})
    return str(h[0].text).replace("\n","").replace("\t", "").replace("\r", "")

In [9]:
def getArticleContent(HTML):
    h = HTML.find_all("div", {"class": "access firstletter"})
    return str(h[0].text).replace("\n","").replace("\t", "").replace("\r", "")

In [10]:
def getArticleAuthor(HTML):
    try:
        h = HTML.find_all("div", {"class": "real-auteur auteur"})
        return str(h[0].text).replace("\n","").replace("\t", "").replace("\r", "")
    except: return ""

In [11]:
def getArticleDate(HTML):
    try:
        h = HTML.find_all("div", {"class": "auteur"})
        date = str(h[0].text).replace("\n","").replace("\t", "").replace("\r", "")
        return date[date.find('le ')+3:date.find('à')-1]
    except: return ""

In [12]:
def getArticleCommentList(URL):
    try:
        driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver',chrome_options=chrome_options)
        driver.get(URL)
        time.sleep(2)
        try:
            element = driver.find_element(by=By.ID, value="disqus_thread").get_attribute('innerHTML')
        finally:
            driver.quit()
        driver.quit()
        p = BeautifulSoup(str(element), "html.parser")
        HTML = getWebPage(p.find('iframe', src=True)['src'])
        data = json.loads(str(HTML.find_all('script', {"id": "disqus-threadData"})[0].text))
        return "|".join([i['raw_message'] for i in data['response']['posts']])
    except:
        return ""

In [13]:
def getLinks(category, index):
    URL = "https://www.leral.net"
    page = getWebPage(f"https://www.leral.net/{category}.html?start={index}")
    _links = page.find_all("div", {"id": "z_col1"})[0].find_all("div", {"class": "cel1"})
    if len(_links) == 0: return []
    _links = [URL+i.find('a', href=True)['href'] for i in _links if isinstance(i.find('a', href=True),bs4.element.Tag)][:-1]
    return _links



def getNewLinks(category, checkPoint):
    links = []
    step = 9
    if category == "Television-Leral_r7" : step = 30
    elif category == "People_r41" : step = 19
    index  = 0
    while(True):
        _links = getLinks(category, index)
        if _links == []:return links
        if checkPoint in _links:
            lastIndex = _links.index(checkPoint)
            if lastIndex == 0:
                return []
            else:
                links.extend(_links)
                return links[:links.index(checkPoint)]
        else:
            links.extend(_links)
            index+=step
    return links     

def getArticle(link,category):
        try:
            p = getWebPage(link)
            _title      = getArticleTitle(p)
            _article    = getArticleContent(p)
            _date       = getArticleDate(p)
            _author     = getArticleAuthor(p)
            _commments  = getArticleCommentList(link)
            return [1,{'DATE_SCRAPING':str(datetime.today()),
                  'DATE':parseDate(_date),
                  'DATE_ARTICLE':_date,
                  'CATEGORY':category,
                  'URL':link,
                  "AUTEUR":_author,
                  "TITRE":_title,
                  "SITE":SITE,
                  "CONTENU":_article,
                  "COMMENTAIRES":_commments,}]
        except:
            return [0,link]


def updateJson(path, key, value):
    if not os.path.exists(path):
        _data = {key:[]}
    else:
        try:
            jsonFile = open(path, "r",encoding='utf-8')
            _data = json.load(jsonFile)
            jsonFile.close()
        except:
            time.sleep(2)
            jsonFile = open(path, "r",encoding='utf-8')
            _data = json.load(jsonFile)
            jsonFile.close()
    _data[key].append(value)
    jsonFile = open(path, "w+", encoding ='utf-8')
    jsonFile.write(json.dumps(_data, default=str))
    jsonFile.close()
    
        
def saveArticle(document):
    Y = document["DATE"].year
    M = document["DATE"].month
    try:
        if not os.path.exists(f"/data/notebooks/DB/DATA/{Y}"):
            os.mkdir(f"/data/notebooks/DB/DATA/{Y}") 
        if not os.path.exists(f"/data/notebooks/DB/DATA/{Y}/{M}"):
            os.mkdir(f"/data/notebooks/DB/DATA/{Y}/{M}")
        return updateJson(f"/data/notebooks/DB/DATA/{Y}/{M}/DATA{ID}.json", "DATA", document)
    except:
        raise Exception(f"{Y}/{M}")


def saveErrors(errList):
    Y = datetime.today().year
    M = datetime.today().month
    return updateJson(f"/data/notebooks/DB/ERRORLOGS/ERRLOGS{ID}.json", f"{Y}{M}", errList)



def checkPoint(category, link=None):
    path = f"/data/notebooks/DB/CHECKPOINT/CP{ID}.json"
    _date = None
    try:
        jsonFile = open(path, "r",encoding='utf-8')
        _data = json.load(jsonFile)
        jsonFile.close()
    except:
        print("ERR")
    if link == None: return _data[category]
    _data[category] = link
    jsonFile = open(path, "w+", encoding ='utf-8')
    jsonFile.write(json.dumps(_data))
    jsonFile.close()
    

    
def update():
    error=[]
    for category in categories:
        _checkPoint = checkPoint(category)
        print(category)
        links = getNewLinks(category, _checkPoint)
        i = 0
        for link in links[::-1]:
            i+=1
            status, document = getArticle(link,category)
            if status==0:error.append(document); continue
            saveArticle(document)
            checkPoint(category,link)
    saveErrors(error)


def initCP():
    path = f"/data/notebooks/DB/CHECKPOINT/CP{ID}.json"
    _data = {}
    for c in categories:
        _data[c] =""
    jsonFile = open(path, "w+", encoding ='utf-8')
    jsonFile.write(json.dumps(_data))
    jsonFile.close()

In [ ]:
update()

In [ ]:
path = "/data/notebooks/DB/DATA/2012/3/DATA002.json"
jsonFile = open(path, "r",encoding='utf-8')
_data = json.load(jsonFile)
jsonFile.close()